Training
This notebook takes the dataset we created in the last notebook and trains the machine learning model.

ACTION: create a folder called pytorchtest in your current working directory.

also have to install pytorch

In [36]:
import numpy as np
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
import glob
import os

import torch
# import torchvision
# import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data as D
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from skimage import measure, filters
from sklearn.preprocessing import OneHotEncoder
print(torch.__version__)
import time
import cv2 as cv
import pickle

save_name = 'pytorchtest/firsttest' 

1.7.1


In [40]:
dirName = ("C:\\SteamCondChamber\\2MLDeepForVision\\CropedImages") #the original and rotated images

In [41]:
def getListOfFiles(dirName):
    # create a list of file and sub directories
    # names in the given directory
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

In [58]:
listOfFiles = getListOfFiles(dirName)
listOfFiles.sort()
L = len(listOfFiles)
n=0
for i in range(L):
    image_name = listOfFiles[i]
    img = cv.imread(image_name)
    #np.reshape(img, (1,32,32))
    #print(img.dtype)
    #print(img.shape)
    #plt.imshow(img)
    #plt.show()
    tensor_im = torch.stack([torch.Tensor(img)])
    dataset = torch.utils.data.TensorDataset(tensor_im)
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=128,shuffle=True)
    #break
    

Initialize CNN Architectures
For a pytorch ML model, the initialization sets up all of the layers that you want available in your model. After each convolutional layer, we renormalize the image (nn.BatchNorm(size)). This normalization step is one of the most important in the whole process!

Parameters to tune when defining a model: Convolutional Layer
A convolutional layer is defined with three main parameters:

the input dimension (how many layers in the input)
the output dimension (how many filters to apply to the input)
the kernel size (how large the trainable kernels or filters should be)
You can also set different values for stride, dilation, and padding, but we did not change these because it changes the shape of the image and can make it more difficult to fit consecutive layers together. The stride is how many pixels the filter moves over every time. Dilation is used to increase the size of the kernel which saves on memory. Lastly, padding adds 0's to the edge of an image.

Input dimension
You can't change this parameter, but you need to set it every time. In the first layer, this will be 1 since a grayscale image has only one layer. In the next convolutional layer, the input dimension will be the output dimension of the previous layer.

Output dimension
You set the number of filters that you want to act on the image at each step. More filters usually give better results, but risks overfitting -- memorizing the data rather than learning important features. Using fewer filters risks underfitting -- learning some features, but not expressive enough to represent all of the data. The model starts with noisy filters and as it learns they become more defined.

Kernel Size
This is also called receptive field. The kernel size determines how much of the image is viewed inside a single filter step. Here we use a 7x7 kernel which means that we use a sliding window of 7x7 pixels to operate on the entire image. Increasing the size of the kernel increases the number of pixels and the size of the area considered at any time. For large, high-resolution images , this is a good thing but, larger kernels require more memory which makes a model more difficult to train. In a deep network (multiple convolutional layers), you can balance what you gain from using more layers with what you gain from increasing the kernel size.

In [59]:
class autoencoder_bottle_large_shallow(nn.Module):
    def __init__(self):
        super(autoencoder_bottle_large_shallow,self).__init__()
        self.conv0 = nn.Conv2d(1,2,7,dilation=1,padding=3)
        self.b0 = nn.BatchNorm2d(2)
        self.conv1 = nn.Conv2d(2,8,7,dilation=1,padding=3)
        self.b1 = nn.BatchNorm2d(8)
        self.conv6 = nn.ConvTranspose2d(8,2,7,stride=2,padding=3)
        self.b6 = nn.BatchNorm2d(2)
        self.conv7 = nn.ConvTranspose2d(2,1,7,stride=2,padding=3)
        self.b7 = nn.BatchNorm2d(1)
        self.pool = nn.MaxPool2d(2,2)

        
    def forward(self,x):
        c0 = self.pool(self.b0(F.relu(self.conv0(x))))
        c1 = self.pool(self.b1(F.relu(self.conv1(c0))))
        c6 = self.b6(F.relu(self.conv6(c1,output_size=c0.size())))
        c7 = self.b7(F.relu(self.conv7(c6,output_size=x.size())))  
        return c7

The `criterion` sets the loss function for the model to use. We use the loss function `MSELoss` for unsupervised learning to see how the output results are "scored" in comparison to the ground truth. MSE creates a criterion that measures the mean squared error (squared L2 norm) between each element in the input and target. 

The parameter `lr` is the learning rate. The learning rate determines how fast the model reacts to any wrong answers. A slower learning rate (lower number) makes smaller adjustments at each step and generally leads to better convergence, but if the learning rate is too slow the model can get stuck in local (not global) minima and it may never converge.

<font color='green'>ACTION (OPTIONAL): Can change learning rate. </font>

<font color='blue'>The line `device = torch.device('cuda:0,1,2,3' if torch.cuda.is_available() else 'cpu')` connects the model to your computer's GPU (graphics card). PyTorch only works with nvidia GPUs, so comment this line out if you don't have one. Similarly, `model = nn.DataParallel(model)` tells python to train using multiple GPUs. This line should be commented out if the computer you are running on only has 1 GPU.</font>

<font color='green'>ACTION (OPTIONAL): The first line of the following cell is where you can choose which model you use. </font>

In [60]:
model = autoencoder_bottle_large_shallow()

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(),lr = 0.0005) 

device = torch.device('cuda:0,1,2,3' if torch.cuda.is_available() else 'cpu') 
print(device)
model.to(device)
model = nn.DataParallel(model) 

cpu


## Training routine

This cell defines the training process for iterating through all of the images and updating the model accordingly.

An epoch is one training iteration through the entire image set. One epoch runs through every batch in the dataset.
The number of epochs determines how long the training process continues. You could set to run as long as the loss is above a certain value.

We usually start with 20 epochs, because this gives good results but is still fast. Once we've tuned all the parameters the best we can, we increase the number of epochs by changing the number in range(). <font color='green'>ACTION (OPTIONAL): Can change number of epochs.</font>

We use `.backward` to determine which steps in the CNN generated significant errors and update the weights using the optimizer. The optimizer we use is called `Adam`. After a batch goes through the network the loss function calculates the differences between the output and ground truth (the input) and this difference goes into the optimizer. The weights are then updated using the optimizer. The goal is for the difference between them (the loss) to be close to 0. 

Ideally, the final plot with the loss value should converge to 0 because we want to minimize the difference between the output and input images. However, in practice, when using an unsupervised autoencoder, we want an imperfect reproduction of the input image. Therefore, the loss value should approach, yet never reach 0.

The example image below shows the segmentation and corresponding intensity histogram we are aiming for during training. 

<font color='blue'>The line `inputs = inputs.to(device)` sends the inputs to the GPU. Remove this if you are not using a GPU.</font> 

In [61]:
loss_history = [] 

start = time.time() 
for epoch in range(3): 

    start_epoch = time.time()
    print('Epoch {}'.format(epoch+1))
    running_loss = 0.0
    n = 0
    
    for i, data in enumerate(dataloader,0):  
        inputs = data[0] 
        inputs = inputs.to(device) 
      
        optimizer.zero_grad()  
        outputs = model(inputs) 

        loss = criterion(outputs,inputs)  
        loss.backward() 
        optimizer.step() 
        
        running_loss += loss.item() 
    
    if (epoch + 1) % 1 == 0:  
        f,a = plt.subplots(ncols=1,figsize=(10,10))
        im = inputs.detach().cpu().numpy()[0,0,:,:]
        out = outputs.detach().cpu().numpy()[0,0,:,:]
#         if (epoch + 1) % 10 == 0:
        a.imshow(im,plt.cm.gray)
        a.contour(out.reshape(512,512),[filters.threshold_otsu(out)])  
        f.savefig(save_name+'_e{}.jpg'.format(epoch+1))
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
        },save_name+'_e{}.pt'.format(epoch+1))
        
        
    print('loss: {}'.format(running_loss/len(dataloader)))
    print('Epoch time: {}\n'.format(time.time() - start_epoch))
    loss_history.append(running_loss/len(dataloader))
        
print('Finished Training')
print('Total Time: {}'.format(time.time() - start))

### Plot loss vs epoch
f,a = plt.subplots(figsize=(10,10)) 
a.set_xlabel('Epoch') 
a.set_ylabel('Loss') 
a.plot(range(len(loss_history)),loss_history) 
f.savefig(save_name+'_Loss100.jpg')

Epoch 1


RuntimeError: Given groups=1, weight of size [2, 1, 7, 7], expected input[1, 32, 32, 3] to have 1 channels, but got 32 channels instead